This is a eMail Spam Classifer that uses Naive Bayes supervised machine learning algorithm.

In [ ]:
#importing necessary packages

import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [ ]:
def dictionary_builder(root_dir):
  words_and_symbols = [] #creating a list for all words and symbols
  all_emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] #reading in the emails into a variable
  for email in all_emails: #looping through each email
    with open(email) as m:
      for line in m:
        word_list = line.split() #splitting each email into a list where each word is a list item
        words_and_symbols += word_list #appending the dictionary to main list
  dictionary = Counter(words_and_symbols) #counting the equivalent values for frequency
  cleaning_list = list(dictionary)

  for character in cleaning_list:
    if character.isalpha() == False: #removing non-alphabet characters
      del dictionary[character]
    elif len(character) == 1: #removing additional symbols and articles
      del dictionary[character]
  dictionary = dictionary.most_common(3000) #returning 3000 most frequent words
  return dictionary
print("List of 3000 most frequent words created") 

List of 3000 most frequent words created


This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.

In [ ]:
def extract_features(mail_dir):
  email_files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(email_files),3000)) #array created
  train_labels = np.zeros(len(email_files)) 
  count = 1;
  docID = 0;
  for file in email_files:
    with open(file) as fi:
      for i, line in enumerate(fi): #adding counter
        if i ==2:
          words = line.split() 
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = file.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels 
print("Label and word frequency matrix generated")               

Label and word frequency matrix generated


The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [ ]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'

In [ ]:
dictionary = dictionary_builder(TRAIN_DIR)

print ("Reading and processing emails from TRAIN and TEST folders\n")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....\n")
model.fit(features_matrix, labels)
print ("Training completed\n")
print ("Testing trained model to predict Test Data labels\n")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:\n")
print (accuracy_score(test_labels, predicted_labels))

Reading and processing emails from TRAIN and TEST folders

Training Model using Gaussian Naibe Bayes algorithm .....

Training completed

Testing trained model to predict Test Data labels

Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:

0.9653846153846154


======================= END OF PROGRAM =========================